In [ ]:
#import all necessary files for capstone project
import sys
import glob
import pandas as pd
import numpy as np
import json
import boto3
import configparser
import datetime
import os

In [ ]:
# Point to the dl.cfg file to get the access and secret keys for reading and writing to S3 on AWS
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

In [ ]:
# Read the AWS access and secret keys
os.environ["AWS_ACCESS_KEY_ID"]=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [ ]:
### Read immigration data for april in SAS format
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immigr_apr = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [ ]:
### convert SAS immigration data to csv and do not include a column for index
df_immigr_apr.to_csv(r'/home/workspace/immigr_apr.csv', index=False)

In [ ]:
# read immigration data in csv format
df_immigr_all = pd.read_csv("/home/workspace/immigr_apr.csv" , low_memory = False )

In [ ]:
# read immigration csv file and replace 'not available' values with empty string
df_immigr_all = pd.read_csv('/home/workspace/immigr_apr.csv', na_values='', dtype=str, low_memory = False)

In [ ]:
# drop columns ( i94cit','count','dtadfile','visapost','occup','entdepa', 'entdepd', 'entdepu',
#'matflag','biryear', 'dtaddto', 'insnum') that are not being used for the data model.
df_immigr2 = df_immigr_all.drop(['i94cit','count','dtadfile','visapost','occup','entdepa', 'entdepd', 'entdepu',
                              'matflag','biryear', 'dtaddto', 'insnum'], axis=1)

In [ ]:
# rename the columns in the immigration dataframe
df_immigr2.rename(columns = {'cicid' : 'cic_id', 'i94yr' : 'year', 'i94mon':'month','i94res':'country','i94port':'us_port',
                            'arrdate': 'arrival_date', 'i94mode' : 'travel_mode', 'i94addr':'us_state', 'depdate':'dep_date',
                            'i94bir': 'age', 'i94visa':'visa_code',
                            'biryear':'birth_year','admnum':'adm_num',
                            'fltno':'flight_num'}, inplace = True )

In [ ]:
# count rows with null values in immigration dataframe
df_immigr2_isnull = df_immigr2[df_immigr2.isnull().any(axis=1)]
df_immigr2_isnull
df_immigr2_isnull.count()

In [ ]:
# drop all null values in the immigration dataframe and count the rows of the final dataframe
df_immigr2.dropna(inplace=True)
df_immigr2.reset_index(drop=True, inplace=True)
df_immigr2.count()

In [ ]:
# list the columns to be converted to integer datatype
cols = ['cic_id', 'year', 'month', 'country', 'arrival_date', 'dep_date', 'age', 'adm_num', 'travel_mode', 'visa_code' ]

In [ ]:
# first convert columns to float
df_immigr2[cols] = df_immigr2[cols].applymap(np.float64)

In [ ]:
# convert columns to integer
df_immigr2[cols] = df_immigr2[cols].applymap(np.int64)

In [ ]:
# Convert arrival date from SAS to date type format
df_immigr2['arrival_date'] = pd.to_timedelta(df_immigr2['arrival_date'], unit='D') + pd.Timestamp('1960-1-1')

In [ ]:
# Convert departure date from SAS to date type format
df_immigr2['dep_date'] = pd.to_timedelta(df_immigr2['dep_date'], unit='D') + pd.Timestamp('1960-1-1')

In [ ]:
# replace visa_code and and travel_mode with the proper code in string value
replace_values_code = { 1.0 : 'Business', 2.0 : 'Pleasure', 3.0 : 'Student' }
replace_values_mode = {1.0 : 'Air', 2.0 : 'Sea' , 3.0 : 'Land', 9.0 : 'Not reported'}
df_immigr3 = df_immigr2.replace({"visa_code": replace_values_code, "travel_mode" : replace_values_mode})                                                                                             
df_immigr3.head() 

In [ ]:
#df_immigr3.head()
#df_immigr3.dtpyes

In [ ]:
df_immigr3.to_csv(r'/home/workspace/immigr_apr_final.csv', index=False)

In [ ]:
# load the immigration data file to s3 bucket
s3 = boto3.client('s3')
s3.upload_file("/home/workspace/immigr_apr_final.csv", 'capstone-bucket-immigr', 'staging_immigr.csv')

In [ ]:
################ CREATE STAGING_DEMO TABLE #################################################

In [ ]:
df_demo1 = pd.read_csv('/home/workspace/us-cities-demographics.csv')

In [ ]:
#header = 'City;State;Median Age;Male Population;Female Population;Total Population;\
#Number of Veterans;Foreign-born;Average Household Size;State Code;Race;Count'
# split the header with semicolon deliminator to columns
df_demo2 = df_demo1['City;State;Median Age;Male Population;Female Population;Total Population;\
Number of Veterans;Foreign-born;Average Household Size;State Code;Race;Count'].str.split(';', 12, expand=True)\
.rename(columns={0:'city', 1:'state_long',  2:'median_age', 3:'male_pop', 4:'female_pop', 5:'total_pop', 6:'veterans',\
                7:'foreign_born', 8:'house_size', 9:'state', 10:'race', 11:'race_count'})

In [ ]:
# drop 'state_long' which will not be used in data model
df_demo2 = df_demo2.drop(columns=['state_long'])

In [ ]:
df_demo2.to_csv(r'/home/workspace/immigr_apr_final.csv', index=False)

In [ ]:
# read demographics csv file with 'not available' values as empty and datatype as string
df_demo2 = pd.read_csv('/home/workspace/demo3_final.csv', na_values='', dtype=str)                    

In [ ]:
# count demographics dataframe null values
demo_isnull = df_demo2[df_demo2.isnull().any(axis=1)]
demo_isnull.count()

In [ ]:
# drop rows with 'not available' values and count the remaining rows
df_demo3 = df_demo2.dropna()
df_demo3.count()

In [ ]:
# list columns that should be integer
cols = ['median_age', 'male_pop','female_pop','total_pop', 'veterans','foreign_born', 'house_size','race_count']

In [ ]:
# first convert column datatypes to float 
df_demo3[cols] = df_demo3[cols].applymap(np.float64)

In [ ]:
# convert columns datatypes to integer
df_demo3[cols] = df_demo3[cols].applymap(np.int64)

In [ ]:
df_demo3.to_json('/home/workspace/demo_final.json')

In [ ]:
staging_demo = pd.read_json('/home/workspace/demo_final.json')

In [ ]:
# use for loop to write each row to a json file in demographics direcotory on local machine
s3 = boto3.client('s3')
for i in staging_demo.index:
    staging_demo.loc[i].to_json("demographics/demo{}.json".format(i))

In [ ]:
# use for loop to load json files to bucket on s3
for i in staging_demo.index:
    s3.upload_file("/home/workspace/demographics/demo{}.json".format(i), 'capstone-bucket-demo', 'demo{}.json'.format(i))